# 1. Load Dataset

In [1]:
import os
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


# Specify the paths
data_dir = '../../../datasets/processed/doom/'

# Load the tensors
variance_tensor = torch.load(os.path.join(data_dir, 'variance_250ms_500hz_tensor.pt'))
labels_tensor = torch.load(os.path.join(data_dir, 'variance_labels_250ms_500hz_tensor.pt'))

# Create the TensorDataset
variance_dataset = torch.utils.data.TensorDataset(variance_tensor, labels_tensor)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(variance_dataset))
val_size = int(0.15 * len(variance_dataset))
test_size = len(variance_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(variance_dataset, [train_size, val_size, test_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

i = 0
# You can now use the DataLoaders in your training loop
for inputs, labels in train_loader:
    # Your training loop here
    print("Total Dataset Size:", len(variance_tensor))
    print("Inputs:", inputs)
    print("Labels:", labels)
    i=i+1
    if i == 1:
        break


Total Dataset Size: 444
Inputs: tensor([[[0.0057],
         [0.0054],
         [0.0045],
         ...,
         [0.0045],
         [0.0140],
         [0.0030]],

        [[0.0026],
         [0.0071],
         [0.0024],
         ...,
         [0.0064],
         [0.0056],
         [0.0088]],

        [[0.0070],
         [0.0050],
         [0.0067],
         ...,
         [0.0029],
         [0.0030],
         [0.0032]],

        ...,

        [[0.0064],
         [0.0059],
         [0.0040],
         ...,
         [0.0017],
         [0.0049],
         [0.0022]],

        [[0.0075],
         [0.0095],
         [0.0021],
         ...,
         [0.0073],
         [0.0063],
         [0.0077]],

        [[0.0136],
         [0.0089],
         [0.0010],
         ...,
         [0.0035],
         [0.0050],
         [0.0096]]])
Labels: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


# Check Shapes

In [2]:
# Print the shape and contents of the variance tensor
print("Variance Tensor Shape:", variance_tensor.shape)
print("Variance Tensor Contents:")
print(variance_tensor)

# Print the shape and contents of the labels tensor
print("Labels Tensor Shape:", labels_tensor.shape)
print("Labels Tensor Contents:")
print(labels_tensor)

Variance Tensor Shape: torch.Size([444, 32, 1])
Variance Tensor Contents:
tensor([[[0.0021],
         [0.0057],
         [0.0020],
         ...,
         [0.0020],
         [0.0030],
         [0.0030]],

        [[0.0024],
         [0.0058],
         [0.0016],
         ...,
         [0.0023],
         [0.0017],
         [0.0013]],

        [[0.0050],
         [0.0069],
         [0.0044],
         ...,
         [0.0017],
         [0.0071],
         [0.0021]],

        ...,

        [[0.0139],
         [0.0063],
         [0.0062],
         ...,
         [0.0017],
         [0.0021],
         [0.0073]],

        [[0.0084],
         [0.0065],
         [0.0106],
         ...,
         [0.0011],
         [0.0066],
         [0.0018]],

        [[0.0040],
         [0.0071],
         [0.0068],
         ...,
         [0.0058],
         [0.0020],
         [0.0052]]])
Labels Tensor Shape: torch.Size([444])
Labels Tensor Contents:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

# Define Custom Activations

In [3]:
import numpy as np
import torch
import torch.nn as nn

class PeakHeightActivation(nn.Module):
    def __init__(self, filepath):
        super(PeakHeightActivation, self).__init__()
        self.peak_height_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        peak_height = torch.tensor(self.peak_height_values, device=x.device)
        return x * peak_height  # Scale by peak height

class PeakCountsActivation(nn.Module):
    def __init__(self, filepath):
        super(PeakCountsActivation, self).__init__()
        self.peak_counts_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        peak_counts = torch.tensor(self.peak_counts_values, device=x.device)
        return x * peak_counts  # Scale by peak counts

class AveragePeakHeightActivation(nn.Module):
    def __init__(self, filepath):
        super(AveragePeakHeightActivation, self).__init__()
        self.average_peak_height_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        average_peak_height = torch.tensor(self.average_peak_height_values, device=x.device)
        return x * average_peak_height  # Scale by average peak height

class AverageDistanceActivation(nn.Module):
    def __init__(self, filepath):
        super(AverageDistanceActivation, self).__init__()
        self.average_distance_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        average_distance = torch.tensor(self.average_distance_values, device=x.device)
        return x * average_distance  # Scale by average distance

class AverageProminenceActivation(nn.Module):
    def __init__(self, filepath):
        super(AverageProminenceActivation, self).__init__()
        self.prominence_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        prominence = torch.tensor(self.prominence_values, device=x.device)
        return x * prominence  # Scale by prominence

class VarianceActivation(nn.Module):
    def __init__(self, filepath):
        super(VarianceActivation, self).__init__()
        self.variance_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        variance = torch.tensor(self.variance_values, device=x.device)
        return x * variance  # Scale by variance

class StdDevActivation(nn.Module):
    def __init__(self, filepath):
        super(StdDevActivation, self).__init__()
        self.std_dev_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        std_dev = torch.tensor(self.std_dev_values, device=x.device)
        return x * std_dev  # Scale by standard deviation

class RMSActivation(nn.Module):
    def __init__(self, filepath):
        super(RMSActivation, self).__init__()
        self.rms_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        rms = torch.tensor(self.rms_values, device=x.device)
        return x * rms  # Scale by RMS

class FrequenciesActivation(nn.Module):
    def __init__(self, filepath):
        super(FrequenciesActivation, self).__init__()
        self.frequencies_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        frequencies = torch.tensor(self.frequencies_values, device=x.device)
        return x * frequencies  # Scale by frequencies

class PSDActivation(nn.Module):
    def __init__(self, filepath):
        super(PSDActivation, self).__init__()
        self.psd_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        psd = torch.tensor(self.psd_values, device=x.device)
        return x * psd  # Scale by PSD

class SpectralEntropyActivation(nn.Module):
    def __init__(self, filepath):
        super(SpectralEntropyActivation, self).__init__()
        self.spectral_entropy_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        spectral_entropy = torch.tensor(self.spectral_entropy_values, device=x.device)
        return x * spectral_entropy  # Scale by spectral entropy

class FFTResultsActivation(nn.Module):
    def __init__(self, filepath):
        super(FFTResultsActivation, self).__init__()
        self.fft_results_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        fft_results = torch.tensor(self.fft_results_values, device=x.device)
        return x * fft_results  # Scale by FFT results

class MagnitudesActivation(nn.Module):
    def __init__(self, filepath):
        super(MagnitudesActivation, self).__init__()
        self.magnitudes_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        magnitudes = torch.tensor(self.magnitudes_values, device=x.device)
        return x * magnitudes  # Scale by magnitudes

class CentroidsActivation(nn.Module):
    def __init__(self, filepath):
        super(CentroidsActivation, self).__init__()
        self.centroids_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        centroids = torch.tensor(self.centroids_values, device=x.device)
        return x * centroids  # Scale by centroids

class SpectralEdgeDensityActivation(nn.Module):
    def __init__(self, filepath):
        super(SpectralEdgeDensityActivation, self).__init__()
        self.spectral_edge_density_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        spectral_edge_density = torch.tensor(self.spectral_edge_density_values, device=x.device)
        return x * spectral_edge_density  # Scale by spectral edge density

class PositiveFrequenciesActivation(nn.Module):
    def __init__(self, filepath):
        super(PositiveFrequenciesActivation, self).__init__()
        self.positive_frequencies_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        positive_frequencies = torch.tensor(self.positive_frequencies_values, device=x.device)
        return x * positive_frequencies  # Scale by positive frequencies

class PositiveFFTResultsActivation(nn.Module):
    def __init__(self, filepath):
        super(PositiveFFTResultsActivation, self).__init__()
        self.positive_fft_results_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        positive_fft_results = torch.tensor(self.positive_fft_results_values, device=x.device)
        return x * positive_fft_results  # Scale by positive FFT results

class CumulativeSumsActivation(nn.Module):
    def __init__(self, filepath):
        super(CumulativeSumsActivation, self).__init__()
        self.cumulative_sums_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        cumulative_sums = torch.tensor(self.cumulative_sums_values, device=x.device)
        return x * cumulative_sums  # Scale by cumulative sums

class TotalPowersActivation(nn.Module):
    def __init__(self, filepath):
        super(TotalPowersActivation, self).__init__()
        self.total_powers_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        total_powers = torch.tensor(self.total_powers_values, device=x.device)
        return x * total_powers  # Scale by total powers

class ThresholdsActivation(nn.Module):
    def __init__(self, filepath):
        super(ThresholdsActivation, self).__init__()
        self.thresholds_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        threshold = torch.tensor(self.thresholds_values, device=x.device)
        return x * threshold  # Scale by thresholds

class PhasesActivation(nn.Module):
    def __init__(self, filepath):
        super(PhasesActivation, self).__init__()
        self.phases_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        phases = torch.tensor(self.phases_values, device=x.device)
        return x * phases  # Scale by phases

class PairwisePhaseLockingValuesActivation(nn.Module):
    def __init__(self, filepath):
        super(PairwisePhaseLockingValuesActivation, self).__init__()
        self.pairwise_phase_locking_values_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        pairwise_phase_locking_values = torch.tensor(self.pairwise_phase_locking_values_values, device=x.device)
        return x * pairwise_phase_locking_values  # Scale by pairwise phase locking values

class HiguchiFractalDimensionActivation(nn.Module):
    def __init__(self, filepath):
        super(HiguchiFractalDimensionActivation, self).__init__()
        self.hfd_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        hfd_values = torch.tensor(self.hfd_values, device=x.device)
        return x * hfd_values  # Scale by Higuchi fractal dimension values
       
class PhaseSpaceActivation(nn.Module):
    def __init__(self, filepath):
        super(PhaseSpaceActivation, self).__init__()
        self.phase_space_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        phase_space = torch.tensor(self.phase_space_values, device=x.device)
        return phase_space

class UMAPActivation(nn.Module):
    def __init__(self, filepath):
        super(UMAPActivation, self).__init__()
        self.umap_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        umap = torch.tensor(self.umap_values, device=x.device)
        return umap

class TSNEActivation(nn.Module):
    def __init__(self, filepath):
        super(TSNEActivation, self).__init__()
        self.tsne_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        tsne = torch.tensor(self.tsne_values, device=x.device)
        return tsne

class SampleEntropyActivation(nn.Module):
    def __init__(self, filepath):
        super(SampleEntropyActivation, self).__init__()
        self.sample_entropy_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        sample_entropy = torch.tensor(self.sample_entropy_values, device=x.device)
        return sample_entropy

class BoxCountingDimensionActivation(nn.Module):
    def __init__(self, filepath):
        super(BoxCountingDimensionActivation, self).__init__()
        self.box_counting_dimension_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        box_counting_dimension = torch.tensor(self.box_counting_dimension_values, device=x.device)
        return box_counting_dimension

class TopologicalActivation(nn.Module):
    def __init__(self, filepath):
        super(TopologicalActivation, self).__init__()
        self.topological_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        topological = torch.tensor(self.topological_values, device=x.device)
        return topological

class LyapunovActivation(nn.Module):
    def __init__(self, filepath):
        super(LyapunovActivation, self).__init__()
        self.lyapunov_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        lyapunov = torch.tensor(self.lyapunov_values, device=x.device)
        return lyapunov

class KatzFractalDimensionActivation(nn.Module):
    def __init__(self, filepath):
        super(KatzFractalDimensionActivation, self).__init__()
        self.katz_fractal_dimension_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        katz_fractal_dimension = torch.tensor(self.katz_fractal_dimension_values, device=x.device)
        return katz_fractal_dimension

class MultiscaleEntropyActivation(nn.Module):
    def __init__(self, filepath):
        super(MultiscaleEntropyActivation, self).__init__()
        self.multiscale_entropy_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        multiscale_entropy = torch.tensor(self.multiscale_entropy_values, device=x.device)
        return multiscale_entropy

class WaveletFractalActivation(nn.Module):
    def __init__(self, filepath):
        super(WaveletFractalActivation, self).__init__()
        self.wavelet_fractal_values = np.memmap(filepath, dtype='float32', mode='r')

    def forward(self, x):
        wavelet_fractal = torch.tensor(self.wavelet_fractal_values, device=x.device)
        return wavelet_fractal


# need to fix the rest of these for memmap npy
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class ApproximateEntropyActivation(nn.Module):
    def __init__(self, m=2, r=0.2):
        super(ApproximateEntropyActivation, self).__init__()
        self.m = m
        self.r = r

    def forward(self, x):
        N = x.size(1)
        phi_m = torch.mean(torch.abs(x[:, None, :N - self.m] - x[:, None, self.m:N]))
        phi_m_plus_one = torch.mean(torch.abs(x[:, None, :N - self.m - 1] - x[:, None, self.m + 1:N]))
        return torch.log(phi_m / (phi_m_plus_one + 1e-9))

class NetworkFractalDimensionActivation(nn.Module):
    def __init__(self):
        super(NetworkFractalDimensionActivation, self).__init__()

    def forward(self, x):
        adjacency_matrix = (x @ x.T) > 0.5
        return torch.log(torch.sum(adjacency_matrix.float())) / torch.log(torch.tensor(adjacency_matrix.size(-1)).float())

class HamiltonianActivation(nn.Module):
    def __init__(self):
        super(HamiltonianActivation, self).__init__()

    def forward(self, x):
        H = x.T @ x - x @ x.T
        return H

class SpectrumAnalysisActivation(nn.Module):
    def __init__(self):
        super(SpectrumAnalysisActivation, self).__init__()

    def forward(self, x):
        spectrum = torch.fft.fft2(x)
        response_surface = torch.abs(spectrum)
        return response_surface

class HarmonicsDetectionActivation(nn.Module):
    def __init__(self):
        super(HarmonicsDetectionActivation, self).__init__()

    def forward(self, x):
        harmonics = torch.fft.fft(x)
        return harmonics.abs()

class HarmonicsLyapunovActivation(nn.Module):
    def __init__(self):
        super(HarmonicsLyapunovActivation, self).__init__()

    def forward(self, x):
        harmonics = torch.fft.fft(x)
        lyapunov_exponent = torch.log(torch.abs(harmonics) + 1e-9)
        return lyapunov_exponent

class ResonanceFrequencyActivation(nn.Module):
    def __init__(self):
        super(ResonanceFrequencyActivation, self).__init__()

    def forward(self, x):
        stft = torch.stft(x, n_fft=128, hop_length=64)
        return torch.abs(stft)

class WeightedUndirectedNetworkActivation(nn.Module):
    def __init__(self):
        super(WeightedUndirectedNetworkActivation, self).__init__()

    def forward(self, x):
        adjacency_matrix = x @ x.T
        return adjacency_matrix

class PhaseSpaceCentroidsActivation(nn.Module):
    def __init__(self):
        super(PhaseSpaceCentroidsActivation, self).__init__()

    def forward(self, x):
        phase_space_2d = torch.stack((x[:, :-1], x[:, 1:]), dim=-1)
        centroids = torch.mean(phase_space_2d, dim=1)
        return centroids

class MultipartiteConcurrenceActivation(nn.Module):
    def __init__(self):
        super(MultipartiteConcurrenceActivation, self).__init__()

    def forward(self, x):
        concurrence = torch.abs(x @ x.T.conj())
        return concurrence

class LaplaceTransformActivation(nn.Module):
    def __init__(self):
        super(LaplaceTransformActivation, self).__init__()

    def forward(self, x):
        s = torch.fft.fft(x)
        return torch.fft.ifft(s / (s + 1e-9))

class SymplecticActivation(nn.Module):
    def __init__(self):
        super(SymplecticActivation, self).__init__()

    def forward(self, x):
        return x @ torch.eye(x.size(1)).to(x.device)

class HyperbolicActivation(nn.Module):
    def __init__(self):
        super(HyperbolicActivation, self).__init__()

    def forward(self, x):
        return torch.sinh(x)

class GeodesicGaussianCurvatureActivation(nn.Module):
    def __init__(self):
        super(GeodesicGaussianCurvatureActivation, self).__init__()

    def forward(self, x):
        curvature = 1 / (1 + torch.sum(x**2, dim=-1))
        return curvature.unsqueeze(-1)

class TheoremaEgregiumActivation(nn.Module):
    def __init__(self):
        super(TheoremaEgregiumActivation, self).__init__()

    def forward(self, x):
        return x / torch.sqrt(1 + torch.sum(x**2, dim=-1)).unsqueeze(-1)

class RiemannianActivation(nn.Module):
    def __init__(self):
        super(RiemannianActivation, self).__init__()

    def forward(self, x):
        return torch.exp(-torch.norm(x, dim=-1)).unsqueeze(-1)

class RiemannCurvatureActivation(nn.Module):
    def __init__(self):
        super(RiemannCurvatureActivation, self).__init__()

    def forward(self, x):
        # Compute the gradient of the input tensor
        gradient = torch.autograd.grad(x.sum(), x, create_graph=True)[0]

        # Compute the Laplacian of the input tensor
        laplacian = sum(torch.autograd.grad(gradient[:, i].sum(), x, create_graph=True)[0][:, i] for i in range(x.shape[1]))

        # Compute the Riemannian Curvature-like quantity
        riemann_curvature = torch.norm(laplacian)

        # Apply a scaling factor to control the magnitude of the curvature
        scaled_curvature = torch.tanh(riemann_curvature)  # You can experiment with different scaling functions

        return scaled_curvature

class JacobianActivation(nn.Module):
    def __init__(self):
        super(JacobianActivation, self).__init__()

    def forward(self, x):
        J = torch.autograd.functional.jacobian(lambda x: x, x)
        return J

class LyapunovVectorsActivation(nn.Module):
    def __init__(self):
        super(LyapunovVectorsActivation, self).__init__()

    def forward(self, x):
        return torch.log(torch.abs(x) + 1e-9)

class LyapunovStabilityActivation(nn.Module):
    def __init__(self):
        super(LyapunovStabilityActivation, self).__init__()

    def forward(self, x):
        return torch.exp(-torch.abs(x))

class LyapunovDimensionActivation(nn.Module):
    def __init__(self):
        super(LyapunovDimensionActivation, self).__init__()

    def forward(self, x):
        return torch.sum(torch.log(1 + torch.abs(x)), dim=-1).unsqueeze(-1)

class OGYActivation(nn.Module):
    def __init__(self):
        super(OGYActivation, self).__init__()

    def forward(self, x):
        return torch.tanh(x)

class KolmogorovSinaiActivation(nn.Module):
    def __init__(self):
        super(KolmogorovSinaiActivation, self).__init__()

    def forward(self, x):
        return torch.log(1 + torch.var(x, dim=-1)).unsqueeze(-1)

class AmplitudeEnvelopeActivation(nn.Module):
    def __init__(self):
        super(AmplitudeEnvelopeActivation, self).__init__()

    def forward(self, x):
        analytic_signal = torch.view_as_complex(torch.fft.fft(x))
        envelope = torch.abs(analytic_signal)
        return envelope

class PyragasActivation(nn.Module):
    def __init__(self):
        super(PyragasActivation, self).__init__()

    def forward(self, x):
        return torch.sin(x) * torch.exp(-torch.abs(x))

class SoulActivation(nn.Module):
    def __init__(self):
        super(SoulActivation, self).__init__()

    def forward(self, x):
        return torch.tanh(x)

class GrowthMeasureActivation(nn.Module):
    def __init__(self):
        super(GrowthMeasureActivation, self).__init__()

    def forward(self, x):
        return torch.exp(-torch.abs(x))

class BernoulliSchemeActivation(nn.Module):
    def __init__(self):
        super(BernoulliSchemeActivation, self).__init__()

    def forward(self, x):
        return torch.sigmoid(x)

class KakutaniActivation(nn.Module):
    def __init__(self):
        super(KakutaniActivation, self).__init__()

    def forward(self, x):
        return torch.cosh(x)

class KakutaniActivation(nn.Module):
    def __init__(self):
        super(KakutaniActivation, self).__init__()

    def forward(self, x):
        return torch.cosh(x)

class MinkowskiActivation(nn.Module):
    def __init__(self):
        super(MinkowskiActivation, self).__init__()

    def forward(self, x):
        return torch.sqrt(torch.abs(x))

class RelativisticVelocityActivation(nn.Module):
    def __init__(self):
        super(RelativisticVelocityActivation, self).__init__()

    def forward(self, x):
        return x / torch.sqrt(1 + x**2)

class SpaceCurvatureActivation(nn.Module):
    def __init__(self):
        super(SpaceCurvatureActivation, self).__init__()

    def forward(self, x):
        return torch.sin(x) / x

class LieGroupActivation(nn.Module):
    def __init__(self):
        super(LieGroupActivation, self).__init__()

    def forward(self, x):
        return torch.exp(x) - 1

class MatrixLieGroupActivation(nn.Module):
    def __init__(self):
        super(MatrixLieGroupActivation, self).__init__()

    def forward(self, x):
        return torch.matrix_exp(x)

class ComplexManifoldActivation(nn.Module):
    def __init__(self):
        super(ComplexManifoldActivation, self).__init__()

    def forward(self, x):
        return torch.view_as_real(x)

class QuaternionicManifoldActivation(nn.Module):
    def __init__(self):
        super(QuaternionicManifoldActivation, self).__init__()

    def forward(self, x):
        return torch.stack((x, -x), dim=-1)

class HermitianManifoldActivation(nn.Module):
    def __init__(self):
        super(HermitianManifoldActivation, self).__init__()

    def forward(self, x):
        return x + x.T.conj()

class BanachManifoldActivation(nn.Module):
    def __init__(self):
        super(BanachManifoldActivation, self).__init__()

    def forward(self, x):
        return torch.norm(x, p=float('inf'))

class FrechetManifoldActivation(nn.Module):
    def __init__(self):
        super(FrechetManifoldActivation, self).__init__()

    def forward(self, x):
        return torch.norm(x, p=2)

class HomomorphismActivation(nn.Module):
    def __init__(self):
        super(HomomorphismActivation, self).__init__()

    def forward(self, x):
        return torch.exp(x)

class InfinitesimalGeneratorActivation(nn.Module):
    def __init__(self):
        super(InfinitesimalGeneratorActivation, self).__init__()

    def forward(self, x):
        return x + torch.eye(x.size(-1)).to(x.device)

class EhresmannConnectionActivation(nn.Module):
    def __init__(self):
        super(EhresmannConnectionActivation, self).__init__()

    def forward(self, x):
        return x * torch.cosh(x)

class VectorBundleActivation(nn.Module):
    def __init__(self):
        super(VectorBundleActivation, self).__init__()

    def forward(self, x):
        return torch.cov(x)

class HolographicInformationActivation(nn.Module):
    def __init__(self):
        super(HolographicInformationActivation, self).__init__()

    def forward(self, x):
        # Apply a 2D Fourier transform to represent holographic information
        hologram = torch.fft.fft2(x)
        
        # Simulate encoding in a lower-dimensional boundary
        boundary_encoding = hologram.mean(dim=-1)
        
        # Use a non-linear transformation to simulate information retrieval
        retrieved_information = torch.tanh(boundary_encoding)
        
        # Transform back to the original domain
        holographic_info = torch.fft.ifft2(retrieved_information)
        
        # Take the real part of the inverse FFT result
        holographic_activation = torch.real(holographic_info)
        
        return holographic_activation

class GaussianActivation(nn.Module):
    def __init__(self, mu=0.0, sigma=1.0):
        super(GaussianActivation, self).__init__()
        self.mu = mu
        self.sigma = sigma

    def forward(self, x):
        return torch.exp(-0.5 * ((x - self.mu) / self.sigma)**2)

class SincActivation(nn.Module):
    def __init__(self):
        super(SincActivation, self).__init__()

    def forward(self, x):
        return torch.where(x == 0, torch.ones_like(x), torch.sin(x) / x)

class PolynomialActivation(nn.Module):
    def __init__(self, coefficients):
        super(PolynomialActivation, self).__init__()
        self.coefficients = coefficients

    def forward(self, x):
        result = torch.zeros_like(x)
        for i, coeff in enumerate(self.coefficients):
            result += coeff * (x ** i)
        return result

class WaveletActivation(nn.Module):
    def __init__(self, wavelet='db1'):
        super(WaveletActivation, self).__init__()
        self.wavelet = wavelet

    def forward(self, x):
        import pywt
        coeffs = pywt.wavedec(x.cpu().numpy(), self.wavelet)
        return torch.tensor(coeffs[0]).to(x.device)

class QuantumActivation(nn.Module):
    def __init__(self, alpha=1.0):
        super(QuantumActivation, self).__init__()
        self.alpha = alpha

    def forward(self, x):
        return torch.sin(self.alpha * x) * torch.exp(-self.alpha * x**2)

class LorentzActivation(nn.Module):
    def __init__(self, c=1.0):
        super(LorentzActivation, self).__init__()
        self.c = c

    def forward(self, x):
        return 1 / torch.sqrt(1 - (x / self.c)**2)

class EntropyActivation(nn.Module):
    def __init__(self, base=2):
        super(EntropyActivation, self).__init__()
        self.base = base

    def forward(self, x):
        p = F.softmax(x, dim=1)
        log_p = torch.log(p + 1e-12) / torch.log(torch.tensor(self.base).float())
        entropy = -torch.sum(p * log_p, dim=1, keepdim=True)
        return x * entropy

class HarmonicOscillatorActivation(nn.Module):
    def __init__(self, omega=1.0):
        super(HarmonicOscillatorActivation, self).__init__()
        self.omega = omega

    def forward(self, x):
        return torch.cos(self.omega * x) * torch.exp(-0.5 * (x / self.omega)**2)

class HarmonicsDetectionActivation(nn.Module):
    def __init__(self):
        super(HarmonicsDetectionActivation, self).__init__()

    def forward(self, x):
        harmonics = torch.fft.fft(x)
        return harmonics.abs()

# CNN Neural Net

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class CustomActivationNet(nn.Module):
    def __init__(self, input_channels=32, input_length=125):
        super(CustomActivationNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        # Initialize custom activations with their respective filepaths
        self.activation1 = PeakHeightActivation(os.path.join(data_dir, 'peak_height_250ms_500hz_tensor.pt'))
        self.activation2 = PeakCountsActivation(os.path.join(data_dir, 'peak_counts_250ms_500hz_tensor.pt'))
        self.activation3 = AveragePeakHeightActivation(os.path.join(data_dir, 'average_peak_height_250ms_500hz_tensor.pt'))
        self.activation4 = AverageDistanceActivation(os.path.join(data_dir, 'average_distance_250ms_500hz_tensor.pt'))
        self.activation5 = AverageProminenceActivation(os.path.join(data_dir, 'average_prominence_250ms_500hz_tensor.pt'))
        self.activation6 = VarianceActivation(os.path.join(data_dir, 'variance_250ms_500hz_tensor.pt'))
        self.activation7 = StdDevActivation(os.path.join(data_dir, 'std_dev_250ms_500hz_tensor.pt'))
        self.activation8 = RMSActivation(os.path.join(data_dir, 'rms_250ms_500hz_tensor.pt'))
        #self.activation9 = FrequenciesActivation(os.path.join(data_dir, 'frequencies_250ms_500hz_tensor.pt'))
        self.activation9 = SpectralEntropyActivation(os.path.join(data_dir, 'spectral_entropy_250ms_500hz_tensor.pt'))
        
        self.fc1 = nn.Linear(256 * (input_length // 8), 512)
        self.fc2 = nn.Linear(512, 1)  # Output layer for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = torch.relu(x)

        x = self.conv2(x)
        x = self.pool2(x)
        x = torch.relu(x)

        x = self.conv3(x)
        x = self.pool3(x)
        x = torch.relu(x)

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                val_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')

# Example training loop
input_size = 32  # Assuming each feature vector has 32 elements (num_channels)
batch_size = 32
num_epochs = 10

for inputs, labels in train_loader:
    print("Input shape:", inputs.shape)
    print("Labels shape:", labels.shape)
    break

# Initialize model, criterion, and optimizer
model = CustomActivationNet(input_channels=input_size, input_length=125)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

# Save the model for future use
torch.save(model.state_dict(), os.path.join(save_dir, 'variance_classifier.pth'))

Input shape: torch.Size([32, 32, 1])
Labels shape: torch.Size([32])


RuntimeError: Given input size: (64x1x1). Calculated output size: (64x1x0). Output size is too small